In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
 %%bash
 module load texlive

In [3]:
import torch
from acctrack.tools.model_inference import TorchModelInference
from acctrack.tools.utils_graph import build_edges
from pathlib import Path

uproot is not installed. AthenaRawRootReader will not be available.


## General Configurations

In [4]:
base_path = "/global/homes/x/xju/m3443/usr/xju/ITk/For2023Paper"
model_path = base_path + "/TrainedModels/v1/graph_construction-MetricLearning-v2.pt"
data_path = base_path + "/2023_uncorr_feature_store"

r_max, k_max = 0.1, 1000
knn_backend = "FAISS-CPU"
config_file = "/pscratch/sd/x/xju/ITk/ForFinalPaper/commonframework/examples/FinalPaper/metric_learning-nonoise.yaml"
embedding_file_name = "embedding.pt"

## Compare different KNN algorithms

### Different algorithms

In [5]:

if Path(embedding_file_name).exists():
    embedding = torch.load(embedding_file_name)
    print(embedding.shape)
else:
    print("Run the embedding creation step first!")


torch.Size([349243, 32])


In [6]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-FlatL2")
edge_index.shape

CPU times: user 1h 11min 30s, sys: 3min 11s, total: 1h 14min 41s
Wall time: 1min 34s


torch.Size([2, 46795462])

In [7]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=1000, nprobe=20)
edge_index.shape

CPU times: user 4min 41s, sys: 55.1 s, total: 5min 36s
Wall time: 5.88 s


torch.Size([2, 42728981])

In [8]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=500, nprobe=20)
edge_index.shape

CPU times: user 6min 6s, sys: 48.3 s, total: 6min 54s
Wall time: 7.26 s


torch.Size([2, 43659143])

In [9]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=50, nprobe=10)
edge_index.shape

CPU times: user 17min 15s, sys: 46.2 s, total: 18min 2s
Wall time: 18.2 s


torch.Size([2, 46041135])

In [10]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=50, nprobe=25)
edge_index.shape

CPU times: user 41min 39s, sys: 48.2 s, total: 42min 27s
Wall time: 43.2 s


torch.Size([2, 46785100])

In [11]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=50, nprobe=30)
edge_index.shape

CPU times: user 49min 35s, sys: 49.1 s, total: 50min 24s
Wall time: 50.3 s


torch.Size([2, 46791439])

In [12]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-scalarquantizer")
edge_index.shape

CPU times: user 2h 47min 15s, sys: 39.3 s, total: 2h 47min 54s
Wall time: 2min 51s


torch.Size([2, 40366750])

In [13]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-GPU")
edge_index.shape

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/global/homes/x/xju/.conda/envs/gnn4itk/lib/python3.9/site-packages/IPython/core/magics/execution.py", line 1325, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
  File "/pscratch/sd/x/xju/ITk/ForFinalPaper/visual_tracks/notebooks/../acctrack/tools/utils_graph.py", line 102, in build_edges
    index_flat = faiss.GpuIndexFlatL2(res, embedding.shape[1])
  File "/global/homes/x/xju/.conda/envs/gnn4itk/lib/python3.9/site-packages/faiss/swigfaiss.py", line 9130, in __init__
    _swigfaiss.GpuIndexFlatL2_swiginit(self, _swigfaiss.new_GpuIndexFlatL2(*args))
RuntimeError: Error in virtual void faiss::gpu::StandardGpuResourcesImpl::initializeForDevice(int) at /project/faiss/faiss/gpu/StandardGpuResources.cpp:283: Error: 'err == cudaSuccess' failed: failed to cudaHostAlloc 268435456 bytes for CPU <-> GPU async copy buffer (error 2 out of memory)

During handling of the above exception, another exception occurred:

Trace

In [16]:
%%time
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FRNN")
edge_index.shape

In [ ]:
%%time
edge_index = build_edges(embedding[:, :3], r_max=r_max, k_max=k_max, backend="FAISS-GPU")
edge_index.shape